<a href="https://colab.research.google.com/github/0xQuirkai/Kaggle-Titanic-Classifier/blob/main/titanic_kaggel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
test_data = pd.read_csv('test.csv')
data = pd.read_csv('train.csv')
gender_submission = pd.read_csv('gender_submission.csv')


# Preprocessing the DATA


In [3]:
def preprocess_data(data):
    data = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Survived']]
    data['Age']=data['Age'].fillna(data['Age'].mean())
    data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})
    data = pd.get_dummies(data, columns=['Embarked'], drop_first=True)
    return data

train = preprocess_data(data)

X, X_test ,Y,y_test = train_test_split(train,train['Survived'], test_size=0.2, random_state=42)


<ipython-input-3-7388b13a6758>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Age']=data['Age'].fillna(data['Age'].mean())
<ipython-input-3-7388b13a6758>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})


## Standardize features by removing the mean and scaling to unit variance.



In [4]:

sc = StandardScaler()
X = sc.fit_transform(X)


# Model

In [5]:



model2 = tf.keras.Sequential([
  tf.keras.layers.Dense(128,activation='relu',input_shape=(X.shape[1],)) ,
  tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64,activation='relu') ,
  tf.keras.layers.Dense(1,activation='sigmoid')

])



# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Assuming you have a model named 'model' and you are training it

# Compile the model
model2.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model2.fit(X, Y, epochs=300, validation_data=(X_test, y_test),verbose=0,callbacks=[early_stopping])

#Evaluate the model on the test set
y_pred = (model2.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')

6/6 [==============================] - 0s 6ms/step
Test Accuracy: 0.8491620111731844


In [6]:

# Combine features with passenger IDs
test_data_with_labels = pd.merge(test_data, gender_submission, on='PassengerId', how='outer')

X_test_new = preprocess_data(test_data_with_labels)

# Make predictions on the preprocessed test data
predictions = model2.predict(X_test_new)

# Convert predictions to binary labels using a threshold (e.g., 0.5)
binary_predictions = (predictions > 0.5).astype(int)

# Create a submission DataFrame
submission_df = pd.DataFrame({'PassengerId': test_data_with_labels['PassengerId'], 'Survived': binary_predictions.flatten()})

# Save the submission file
submission_df.to_csv('my_submission.csv', index=False)


<ipython-input-3-7388b13a6758>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Age']=data['Age'].fillna(data['Age'].mean())
<ipython-input-3-7388b13a6758>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})


14/14 [==============================] - 0s 2ms/step


In [7]:

# Read and display the contents
my_submission = pd.read_csv('my_submission.csv')

# Merge DataFrames on PassengerId
merged_df = pd.merge(gender_submission, my_submission, on='PassengerId', suffixes=('_actual', '_predicted'))
# Calculate accuracy
accuracy = (merged_df['Survived_actual'] == merged_df['Survived_predicted']).mean()

print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 76.32%


In [8]:
from google.colab import files

# Specify the file path
file_path = 'my_submission.csv'

# Download the file
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>